# SQI Clustering using DBSCAN

The DBSCAN algorithm views clusters as areas of high density separated by
areas of low density. Due to this rather generic view, clusters found by
DBSCAN can be any shape, as opposed to k-means which assumes that clusters
are convex shaped.

In [ ]:
# Libraries
# Generic
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Scipy
from scipy.stats import skew
from scipy.stats import kurtosis
from scipy.stats import entropy

# Scikit-learn
from sklearn.cluster import DBSCAN

# 3D projection
from mpl_toolkits.mplot3d import Axes3D

# Heartpy
import heartpy as hp

# Plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# vitalSQI
from vital_sqi.data.signal_io import ECG_reader
from vital_sqi.data.signal_io import PPG_reader
from vital_sqi.dataset import load_ppg, load_ecg

TERMINAL = True

# Data path
filepath = 'D:/IC Final Year Project/01NVa-003-2001/PPG'
filename = '01NVa-003-2001 Smartcare.csv'

# Load patient data
signals = pd.read_csv(os.path.join(filepath, filename),
                      nrows=200000)

# Calculate sampling rate
fs = 1 / (signals.TIMESTAMP_MS.diff().median() * 0.001)

# Datetime start
start_datetime = '2021-01-01 17:00:00'

# Format the data

# Display (shows timedelta aligned)
pd.Timedelta.__str__ = lambda x: x._repr_base('all')

# Include column with index
signals = signals.reset_index()

signals['timedelta'] = \
    pd.to_timedelta(signals.TIMESTAMP_MS, unit='ms')

# Create datetimes (if needed)
signals['date'] = pd.to_datetime(start_datetime)
signals['date'] += pd.to_timedelta(signals.timedelta)

# Set the timedelta index (keep numeric index too)
signals = signals.set_index('timedelta')

# Rename column index to avoid confusion
signals = signals.rename(columns={'index': 'idx'})

# Create figure
# fig = go.Figure(go.Scatter(
#     x=signals.date,
#     y=signals.IR_ADC,
#     name='IR_ADC'))
# fig.update_xaxes(rangeslider_visible=True)
# fig.show() # Uncomment if running locally

# Trim first/last 5 minutes
# Offset
offset = pd.Timedelta(minutes=5)

# Indexes
idxs = (signals.index > offset) & \
       (signals.index < signals.index[-1] - offset)

# Filter
signals = signals[idxs]

def bbpf_rep(s):
    """Butter Band Pass Filter Scipy

    This is equivalent to the BandPassFilter() class implemented
    in the vital_sqi package. Need to understand the difference
    between filtfilt and lfilter.

    There is no need to compute the nyquist frequency. We can
    just pass the frequency sample and scipy will adjust the
    cutoff frequencies according to the nyquist theorem:

                f = f / 0.5*fs

    Note that scipy has also a bandpass filter (btype=band) but
    it is not possible to choose different orders for the high
    pass and low pass filters.
    """
    from scipy import signal
    # Configure high/low filters
    bh, ah = signal.butter(1, 1, fs=fs, btype='high', analog=False)
    bl, al = signal.butter(4, 20, fs=fs, btype='low', analog=False)
    # Apply filters
    aux = signal.filtfilt(bh, ah, s)
    aux = signal.lfilter(bl, al, aux)
    # Return
    return aux


# Add vital signals
signals['PLETH_BPF'] = bbpf_rep(signals.PLETH)

# Show
# print("\nSignals (all):")
# signals
#
# if TERMINAL:
#     print(signals)

# Compute SQIs
import vital_sqi.sqi as sq


def snr(x, axis=0, ddof=0):
    """Signal to noise ratio"""
    a = np.asanyarray(x)
    m = a.mean(axis)
    sd = a.std(axis=axis, ddof=ddof)
    return np.where(sd == 0, 0, m / sd)


def zcr(x):
    """Zero crossing rate"""
    return 0.5 * np.mean(np.abs(np.diff(np.sign(x))))


def mcr(x):
    """Mean crossing rate"""
    return zcr(x - np.mean(x))


def perfusion(x, y):
    """Perfusion

    Parameters
    ----------
    x: raw signal
    y: filtered signal
    """
    return (np.max(y) - np.min(y)) / np.abs(np.mean(x)) * 100


def correlogram(x):
    """Correlogram"""
    return sq.rpeaks_sqi.correlogram_sqi(x)


def msq(x):
    """Mean signal quality (is this the acronym?)

    Feedback for the package:
      The msq_sqi should also allow to receive two peak detector
      ids as input parameters. At the moment only allows a list
      of peaks and a peak detector id.
    """
    # Library
    from vital_sqi.common.rpeak_detection import PeakDetector
    # Detection of peaks
    detector = PeakDetector()
    peak_list, trough_list = detector.ppg_detector(x, 7)
    # Return
    return sq.standard_sqi.msq_sqi(x, peaks_1=peak_list, peak_detect2=6)


def dtw(x):
    """Dynamic time warping

    .. note: It is very slow!!

    Returns
    -------
    [mean, std]
    """
    # Library
    from vital_sqi.common.rpeak_detection import PeakDetector
    # Detection of peaks
    detector = PeakDetector()
    peak_list, trough_list = detector.ppg_detector(x, 7)
    # Per beats
    dtw_list = sq.standard_sqi.per_beat_sqi( \
        sqi_func=sq.dtw_sqi, troughs=trough_list,
        signal=x, taper=True, template_type=1
    )
    # Return mean
    return [np.mean(dtw_list),
            np.std(dtw_list)]


def sqi_all(x):
    """Compute all SQIs.

    .. note: If some variables are required for different
             SQIS (e.g. detecting the peaks), it might be
             possible to save some time by computing them
             first instead of having each method to compute
             them within the method. There is a trade off
             between the efficiency gained (depends on the
             implementation of the PeakDetector) and the
             the simplicity of the code and usage.
    """
    # Information
    dinfo = {
        'first': x.idx.iloc[0],
        'last': x.idx.iloc[-1],
        'skew': skew(x['PLETH']),
        'kurtosis': kurtosis(x['PLETH']),
        'snr': snr(x['PLETH']),
        'mcr': mcr(x['PLETH']),
        'zcr': zcr(x['PLETH_BPF']),
        'msq': msq(x['PLETH_BPF']),
        'perfusion': perfusion(x['PLETH'], x['PLETH_BPF']),
        'correlogram': correlogram(x['PLETH_BPF']),
        # 'dtw': dtw(x['0_bbpf'])
    }

    # Return
    return pd.Series(dinfo)

# 1. use the method 'agg'
# Group by 30s windows/aggregate
sqis = signals \
    .groupby(pd.Grouper(freq='30s')) \
    .agg({'idx': ['first', 'last'],
          'PLETH': [skew, kurtosis, snr, mcr],
          'IR_ADC': [skew, kurtosis, snr, mcr],
          'PLETH_BPF': [zcr, msq, correlogram]  # dtw]
          })

# .. note: We are assuming that the whole signal has been
#          read in one chunk. This will not work if using
#          batches, will window ids be necessary?
# Add window id (if needed)
sqis['w'] = np.arange(sqis.shape[0])

# Show
# print("\nSQIs (agg):")
# sqis
#
# if TERMINAL:
#     print(sqis)

# 2. use 'apply' to compare
# Group by 30s windows/apply
sqis2 = signals \
    .groupby(pd.Grouper(freq='30s')) \
    .apply(sqi_all)

# Show
# print("\nSQIs (apply):")
# sqis2
#
# if TERMINAL:
#     print(sqis2)

print('Data points used for clustering: sqis.PLETH')
print(sqis.PLETH)